In [5]:
# Libraries import
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import gym
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from transformers import TFGPT2LMHeadModel

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
# Environment architecture
class ChatbotEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, messages):
        self.action_space = gym.spaces.Discrete(2) # Two possible actions: respond with a fixed message or ask a follow-up question
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(len(messages),)) # Binary vector representing the user's message
        self.messages = messages
        self.state = None

    def step(self, action):
        if action == 0: # Respond with a fixed message line
            response = "Thank you for your message. Our team will get back to you soon."
            reward = 1
            done = True
        else: # Ask a follow-up question from user
            response = "Can you provide more information about your request?"
            reward = 0
            done = False

        return self.state, reward, done, {'response': response}

    def reset(self):
        self.state = np.zeros(len(self.messages))
        return self.state

class ChatbotAgent:
    def __init__(self, env, model, tokenizer):
        self.env = env
        self.model = TFGPT2LMHeadModel.from_pretrained("gpt2")
        self.tokenizer = Tokenizer()

    def train(self, episodes):
        max_steps_per_episode = 1
        for episode in range(episodes):
            # Reset the environment for a new episode
            observation = self.env.reset()

            # Implement your training logic here
            # For example, loop through steps, choose actions, and update the model
            for step in range(max_steps_per_episode):
                action = self.select_action(observation)
                next_observation, reward, done, info = self.env.step(action)
                observation = next_observation

                if done:
                    break

    def tokenize(self, text):
        # Use NLTK word_tokenize for tokenization
        return word_tokenize(text)

    def select_action(self, observation):
        # Tokenize the elements of observation
        observation = [self.tokenize(str(item)) for item in observation]

        # Flatten the list of tokenized words into a single list
        observation = [word for sublist in observation for word in sublist]
        self.tokenizer.fit_on_texts(observation)

        # Convert the observation (list of tokens) to a single string
        input_text = ' '.join(observation)

        # Tokenize the input text
        input_sequence = self.tokenizer.texts_to_sequences([' '.join(observation)])[0]

        # Pad the sequence to a fixed length
        input_sequence = pad_sequences([input_sequence], maxlen=50, padding='post')
        with tf.device("/gpu:0"):  # Use GPU if available
            output = self.model.generate(input_sequence, max_length=1050, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95)

        # Decode the generated response
        response = self.tokenizer.sequences_to_texts(output.numpy())[0]


        # Decide on the action based on the generated response (you can define your own logic)
        action = 0 if "positive" in response.lower() else 1

        return action

In [3]:
# this step will Collect and preprocess data
messages = ['How can I help you?', 'Can you provide more information?', 'Thank you for your message.']
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Load the chat logs
chat_logs = []
with open('/content/drive/MyDrive/Colab_Notebooks/chat_log.txt', 'r') as f:
    for line in f:
        line = line.strip().lower()
        if line:
            chat_logs.append(line)

# Tokenize and preprocess the chat messages
tokenized_messages = []
for message in messages:
    tokens = word_tokenize(message.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    tokenized_messages.append(' '.join(tokens))

# Tokenize and preprocess chat logs through the below code.
tokenized_logs = []
for log in chat_logs:
    tokens = word_tokenize(log)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    tokenized_logs.append(' '.join(tokens))

# Create a tokenizer and fit it on tokenized messages.
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_messages)

# Convert tokenized log to sequences
sequences = tokenizer.texts_to_sequences(tokenized_logs)

# Pad sequences to ensure each sequences have the same length.
max_len = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

In [12]:
# This step will create a chatbot environment and agent.
model = TFGPT2LMHeadModel.from_pretrained("gpt2")
env = ChatbotEnv(tokenized_messages)
agent = ChatbotAgent(env, model, tokenizer)

# Train the agent with the a train() function.
agent.train(episodes=1)

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
# Test the created chatbot
while True:
    user_input1 = input("User: ")
    tokens = word_tokenize(user_input1.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    tokenized_input = ' '.join(tokens)
    sequence = tokenizer.texts_to_sequences([tokenized_input])
    padded_sequence = pad_sequences(sequence, maxlen=max_len, padding='post')
    action = agent.select_action(padded_sequence[0])
    response = env.step(action)[3]['response']
    print("Chatbot : " + response) # Test the chatbot

while True:
    user_input1 = input("User: ")
    tokens = word_tokenize(user_input1.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    tokenized_input = ' '.join(tokens)
    sequence = tokenizer.texts_to_sequences([tokenized_input])
    padded_sequence = pad_sequences(sequence, maxlen=max_len, padding='post')
    action = agent.select_action(padded_sequence[0])
    response = env.step(action)[3]['response']
    print("Chatbot response: " + response)

User: What is chat?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot : Can you provide more information about your request?
User: online communication between two person


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot : Can you provide more information about your request?
User: no


...

Chatbot : Can you provide more information about your request?
User: Yes


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot : Can you provide more information about your request?
User: No, Welcome


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot : Can you provide more information about your request?
